# **Abrir csv y empezar a leer lo basico**
***

## Librerias a ocupar
***

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import json
import re

### Abrir el archivo
***

In [2]:
with open("data/games_march2025_cleaned.csv", "r", encoding='utf-8') as archivo:
    data = pd.read_csv(archivo)

### Informacion basica de la data
***

In [3]:
data.head()

,appid,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,879,5174,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,1536,898,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,771,7101,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,682,2434,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608


In [9]:
data.columns

Index(['appid', 'name', 'release_date', 'required_age', 'price', 'dlc_count',
       'detailed_description', 'about_the_game', 'short_description',
       'reviews', 'header_image', 'website', 'support_url', 'support_email',
       'windows', 'mac', 'linux', 'metacritic_score', 'metacritic_url',
       'achievements', 'recommendations', 'notes', 'supported_languages',
       'full_audio_languages', 'packages', 'developers', 'publishers',
       'categories', 'genres', 'screenshots', 'movies', 'user_score',
       'score_rank', 'positive', 'negative', 'estimated_owners',
       'average_playtime_forever', 'average_playtime_2weeks',
       'median_playtime_forever', 'median_playtime_2weeks', 'discount',
       'peak_ccu', 'tags', 'pct_pos_total', 'num_reviews_total',
       'pct_pos_recent', 'num_reviews_recent'],
      dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89618 entries, 0 to 89617
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   appid                     89618 non-null  int64  
 1   name                      89618 non-null  object 
 2   release_date              89618 non-null  object 
 3   required_age              89618 non-null  int64  
 4   price                     89618 non-null  float64
 5   dlc_count                 89618 non-null  int64  
 6   detailed_description      89421 non-null  object 
 7   about_the_game            89398 non-null  object 
 8   short_description         89498 non-null  object 
 9   reviews                   10401 non-null  object 
 10  header_image              89618 non-null  object 
 11  website                   41114 non-null  object 
 12  support_url               44110 non-null  object 
 13  support_email             78798 non-null  object 
 14  window

In [33]:
# Eliminar algunas columnas para limpiar
# Elimino todas las columnas para quedarme solo con las cosas positivas
columnas_eliminar = ["detailed_description", "about_the_game", "short_description",
                      "reviews", "header_image", "website", "support_url", "support_email",
                      "metacritic_url", "achievements", "notes", "screenshots", "movies",
                      "tags", 'required_age', 'price', 'dlc_count',
                      'windows', 'mac', 'linux','supported_languages',
                      'full_audio_languages', 'packages', 'developers', 'publishers',
                      'categories', 'genres', 'estimated_owners',
                      'average_playtime_forever', 'average_playtime_2weeks',
                      'median_playtime_forever', 'median_playtime_2weeks', 'discount',
                      'peak_ccu', 'tags', "user_score", "score_rank", "metacritic_score"
                      ]

df_reseñas = data.drop(columns=columnas_eliminar)
df_reseñas.head()

,appid,name,release_date,recommendations,positive,negative,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,4401572,7480813,1135108,86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,1732007,1487960,1024436,59,2513842,68,16720
2,570,Dota 2,2013-07-09,14337,1998462,451338,81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,2015-04-13,1803063,1719950,250012,87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,1165929,1152763,218446,84,1168020,76,12608


## Primero agrupar los juegos con sus secuelas y ver si estos son bien resividos mediante las reseñas

**Dado que las escala de metacritic_score no siempre esta actualizada o no tiene los datos lo voy a eliminar**

**- Asique me voy a quedar con solo las reseñas de steam que son solo usuarios**

In [ ]:
# 1.- Mucha limpieza

# Les cambio el nombre a las columnas para mayor comodidad
df_reseñas_limpio = df_reseñas_limpio.rename(columns={"appid": "id",
                                    'name': 'Nombre',
                                  'recommendations' : 'Recomendaciones',
                                  "positive" : "Reseñas_positivas",
                                  "negative" : "Reseñas_negativas",
                                  "pct_pos_total" : "Porcentaje_Positivo_Total",
                                  "num_reviews_total" : "Nun_total_de_reseñas",
                                  "pct_pos_recent" : "Porcentaje_positivo_reciente",
                                  "num_reviews_recent" : "Numero_de_reseñas_recientes",
                                  "release_date" : "Fecha_de_lanzamiento"}
                                  )
                   
df_reseñas_limpio['Fecha_de_lanzamiento'] = pd.to_datetime(df_reseñas_limpio['Fecha_de_lanzamiento'], errors='coerce')

# Elimino las columnas que deben ser claves
df_reseñas_limpio = df_reseñas_limpio.dropna(subset=["Recomendaciones", "Nun_total_de_reseñas",
                                                    'Fecha_de_lanzamiento', 'Reseñas_positivas',
                                                    "Reseñas_positivas", "Reseñas_negativas",
                                                    "Porcentaje_Positivo_Total"])





In [62]:
# Hacer una funcion para encontrar una columna de nombre en comun
def obtener_nombre_base(titulo):
    # Si el título no es un string, devuélvelo tal cual
    if not isinstance(titulo, str):
        return titulo
    
    # 1. Quitar subtítulos (ej. "Counter-Strike: Global Offensive" -> "Counter-Strike")
    base = re.sub(r':.*', '', titulo)
    
    # 2. Quitar texto en paréntesis o corchetes (ej. "DOOM (2016)" -> "DOOM")
    base = re.sub(r'\s*\(.*\)', '', base)
    base = re.sub(r'\s*\[.*\]', '', base)
    
    # 3. Quitar números arábigos o romanos al final (ej. "Half-Life 2" -> "Half-Life")
    # (Busca un espacio seguido de números o letras I, V, X al final de la línea)
    base = re.sub(r'\s+([IVX\d]+)$', '', base)
    
    # 4. Quitar palabras comunes de "edición"
    base = re.sub(r'\s+(GOTY|Remastered|Definitive Edition)$', '', base, flags=re.IGNORECASE)
    
    return base.strip()

# Aplicamos la función a toda la columna 'Nombre'
df_reseñas_limpio['Nombre_Base'] = df_reseñas_limpio['Nombre'].apply(obtener_nombre_base)

# Contamos cuántos juegos hay en cada grupo de 'Nombre_Base'
conteo_franquicias = df_reseñas_limpio.groupby('Nombre_Base')['id'].count()

# Obtenemos la lista de nombres base que SÍ son franquicias (tienen > 1 juego)
nombres_franquicias = conteo_franquicias[conteo_franquicias > 1].index

# Filtramos nuestro DataFrame para quedarnos SÓLO con los juegos que pertenecen a una franquicia
df_franquicias = df_reseñas_limpio[df_reseñas_limpio['Nombre_Base'].isin(nombres_franquicias)]



df_franquicias.head()

,id,Nombre,Fecha_de_lanzamiento,Recomendaciones,Reseñas_positivas,Reseñas_negativas,Porcentaje_Positivo_Total,Nun_total_de_reseñas,Porcentaje_positivo_reciente,Numero_de_reseñas_recientes,Nombre_Base
0,730,Counter-Strike 2,2012-08-21,4401572,7480813,1135108,86,8632939,82,96473,Counter-Strike
10,2358720,Black Myth: Wukong,2024-08-19,825294,1098373,37811,96,825621,94,6139,Black Myth
14,1091500,Cyberpunk 2077,2020-12-09,735210,690253,132012,85,729016,96,7612,Cyberpunk
16,1086940,Baldur's Gate 3,2023-08-03,647659,709860,23273,96,648637,96,8010,Baldur's Gate
17,227300,Euro Truck Simulator 2,2012-10-12,669576,825685,22349,97,641347,96,8950,Euro Truck Simulator


In [75]:
# Ordenamos por nombre de franquicia y luego por fecha
df_franquicias = df_franquicias.sort_values(by=['Nombre_Base', 'Fecha_de_lanzamiento'])

# 1. Creamos un DataFrame SÓLO con los originales
# (El primer juego de cada grupo)
df_originales = df_franquicias.groupby('Nombre_Base').first().reset_index()

# 2. Creamos un DataFrame SÓLO con las secuelas
# (Cualquier juego que NO esté en nuestra lista de 'appid' originales)
ids_originales = df_originales['id']
df_secuelas = df_franquicias[~df_franquicias['id'].isin(ids_originales)]

df_secuelas.sample(10)

,id,Nombre,Fecha_de_lanzamiento,Recomendaciones,Reseñas_positivas,Reseñas_negativas,Porcentaje_Positivo_Total,Nun_total_de_reseñas,Porcentaje_positivo_reciente,Numero_de_reseñas_recientes,Nombre_Base
32802,353110,Dreamscapes: Nightmare's Heir - Premium Edition,2015-03-11,0,120,53,76,34,-1,-1,Dreamscapes
33731,698620,Screamer 2,2017-09-15,0,24,7,77,31,-1,-1,Screamer
20983,2298690,Realpolitiks 3: Earth and Beyond,2024-10-22,0,63,48,58,93,-1,-1,Realpolitiks
35704,2777030,In The Shadows,2024-01-29,0,0,0,55,27,-1,-1,In The Shadows
19878,2327480,Backrooms: Realm of Shadows,2023-07-11,0,51,51,49,105,-1,-1,Backrooms
45305,2197780,Nash Racing: Pursuit,2022-11-30,0,14,11,66,15,-1,-1,Nash Racing
16304,2063530,Methods: The Canada Files,2023-02-01,158,169,7,95,157,-1,-1,Methods
56603,2200330,Project: Nightlight,2023-03-10,0,1,4,-1,-1,-1,-1,Project
59688,3240300,Elohim Eternal: Exodus,2025-01-13,0,0,0,-1,-1,-1,-1,Elohim Eternal
65418,2182390,Pandamonium,2023-06-01,0,5,0,-1,-1,-1,-1,Pandamonium


In [76]:
# Unimos secuelas con sus respectivos originales
df_comparativo = pd.merge(
    df_secuelas,
    df_originales,
    on='Nombre_Base',
    suffixes=('_Secuela', '_Original') # ¡Importante para diferenciar columnas!
)

# Ahora df_comparativo tiene columnas como 'Nombre_Secuela', 'Nombre_Original',
# 'Reseñas_positivas_Secuela', 'Reseñas_positivas_Original', etc.

In [77]:
# 1. Seleccionamos solo las columnas que nos interesan para la comparativa
columnas_finales = [
    'Nombre_Base',
    'Nombre_Original',
    'Reseñas_positivas_Original',
    'Reseñas_negativas_Original',
    'Nombre_Secuela',
    'Reseñas_positivas_Secuela',
    'Reseñas_negativas_Secuela'
]
df_final = df_comparativo[columnas_finales]

# 2. Creamos columnas de diferencia para ver el éxito
df_final['Diferencia_Positivas'] = df_final['Reseñas_positivas_Secuela'] - df_final['Reseñas_positivas_Original']
df_final['Diferencia_Negativas'] = df_final['Reseñas_negativas_Secuela'] - df_final['Reseñas_negativas_Original']

# 3. Vemos las secuelas que MÁS mejoraron las reseñas positivas
print("Secuelas más exitosas (comparadas con el original):")
print(df_final.sort_values(by='Diferencia_Positivas', ascending=False).head(10))

# 4. Vemos las secuelas que MÁS empeoraron (más reseñas negativas)
print("\nSecuelas que más decepcionaron (más reseñas negativas):")
print(df_final.sort_values(by='Diferencia_Negativas', ascending=False).head(10))

Secuelas más exitosas (comparadas con el original):
               Nombre_Base                              Nombre_Original  \
1251        Counter-Strike                       Counter-Strike: Source   
685             Black Myth            Black Myth: Wukong Benchmark Tool   
579          Baldur's Gate              Baldur's Gate: Enhanced Edition   
1358             Cyberpunk                               Cyberpunk 3776   
877          Call of Duty®                     Call of Duty®: Black Ops   
1368           DARK SOULS™                               DARK SOULS™ II   
727              Bloons TD                                  Bloons TD 5   
2160               Fallout  Fallout 2: A Post Nuclear Role Playing Game   
2772        Hearts of Iron                Hearts of Iron III: Semper Fi   
6108  The Binding of Isaac                         The Binding of Isaac   

      Reseñas_positivas_Original  Reseñas_negativas_Original  \
1251                      172801                        66

C:\Users\reyes\AppData\Local\Temp\ipykernel_35880\2187914252.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Diferencia_Positivas'] = df_final['Reseñas_positivas_Secuela'] - df_final['Reseñas_positivas_Original']
C:\Users\reyes\AppData\Local\Temp\ipykernel_35880\2187914252.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Diferencia_Negativas'] = df_final['Reseñas_negativas_Secuela'] - df_final['Reseñas_negativas_Original']
